<a href="https://colab.research.google.com/github/dibab64/deeplearning-dibab/blob/main/voc2007.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dataset downloaded as `pascal-voc-2007.zip`

In [2]:
from google.colab import userdata
userdata.get('furkansoyupak')

'629d247b92b8bdd63d8ea85cfe54fecd'

In [6]:
!mkdir /root/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [7]:
!mv /content/kaggle.json /root/.kaggle/kaggle.json

In [10]:
!chmod 600 /root/.kaggle/kaggle.json

In [11]:
!kaggle datasets download -d zaraks/pascal-voc-2007

... resuming from 250609664 bytes (1524241964 bytes left) ...
100% 1.65G/1.65G [00:52<00:00, 35.6MB/s]
100% 1.65G/1.65G [00:52<00:00, 29.1MB/s]


In [ ]:
!unzip pascal-voc-2007.zip

Görüntülenen çıkış son 5000 satıra kısaltıldı.
  inflating: VOCtrainval_06-Nov-2007/VOCdevkit/VOC2007/Annotations/003636.xml  
  inflating: VOCtrainval_06-Nov-2007/VOCdevkit/VOC2007/Annotations/003638.xml  
  inflating: VOCtrainval_06-Nov-2007/VOCdevkit/VOC2007/Annotations/003639.xml  
  inflating: VOCtrainval_06-Nov-2007/VOCdevkit/VOC2007/Annotations/003640.xml  
  inflating: VOCtrainval_06-Nov-2007/VOCdevkit/VOC2007/Annotations/003642.xml  
  inflating: VOCtrainval_06-Nov-2007/VOCdevkit/VOC2007/Annotations/003644.xml  
  inflating: VOCtrainval_06-Nov-2007/VOCdevkit/VOC2007/Annotations/003645.xml  
  inflating: VOCtrainval_06-Nov-2007/VOCdevkit/VOC2007/Annotations/003646.xml  
  inflating: VOCtrainval_06-Nov-2007/VOCdevkit/VOC2007/Annotations/003648.xml  
  inflating: VOCtrainval_06-Nov-2007/VOCdevkit/VOC2007/Annotations/003651.xml  
  inflating: VOCtrainval_06-Nov-2007/VOCdevkit/VOC2007/Annotations/003654.xml  
  inflating: VOCtrainval_06-Nov-2007/VOCdevkit/VOC2007/Annotations/003655